In [1]:
from pyserini.search.lucene import LuceneSearcher
import re
import json
import torch
from sentence_transformers import SentenceTransformer, util

/home/kjros2/.local/lib/python3.8/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [14]:
#exp_queries = "51-100"
exp_queries = "covid"

if exp_queries == "51-100":
    qrel_path = "src/main/resources/topics-and-qrels/qrels.adhoc.51-100.txt"
    query_path = "src/main/resources/topics-and-qrels/topics.adhoc.51-100.txt"
    index_path = "indexes/lucene-index.disk12"
elif exp_queries == "covid":
    qrel_path = "src/main/resources/topics-and-qrels/qrels.covid-round1.txt"
    query_path = "src/main/resources/topics-and-qrels/topics.covid-round1.xml"
    index_path = "indexes/lucene-index-cord19-abstract-2020-07-16/"
    
searcher = LuceneSearcher(index_path)
    

In [15]:
# query_id : raw query words
all_query_words = {}
# query_id : [relevant document ids]
all_qrels = {}

if exp_queries == "51-100":
    with open(query_path, "r") as f:
        current_id = None
        for line in f:
            if "Number:" in line:
                current_id = str(int(line.split()[2]))
                all_query_words[current_id] = []
            if "Topic:" in line:
                query = line.split()[2:]
                all_query_words[current_id] = " ".join(query)

elif exp_queries == "covid":
    with open(query_path, "r") as f:
        for line in f:
            if "<topic number=" in line:
                current_id = re.sub("[^0-9]", "", line)
                all_query_words[current_id] = []
            if "<query>" in line:
                line = re.sub("<query>", "", line)
                line = re.sub("</query>", "", line)
                line = re.sub("\n", "", line)
                all_query_words[current_id] = line

with open(qrel_path, "r") as f:
    for line in f:
        query_id, _, doc_id, _ = line.split()
        if query_id not in all_qrels:
            all_qrels[query_id] = []
        all_qrels[query_id].append(doc_id)

#### Below is for optimal approach #2: Contextual expansion using relevance judgments

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/kjros2/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [ ]:
# make function that encodes document, encodes query, and findest most similar latter tokens to former token

lower_bound = 0.5
upper_bound = 0.7
considered_k = 50
max_k = 5

expansion_terms = {}
for q,query_id in enumerate(all_query_words):
    print(query_id)
    query_text = all_query_words[query_id].lower()
    query_text = re.sub("[^a-z0-9 ]", " ", query_text)
    print(query_text)
    doc_id = all_qrels[query_id][0]
    doc = searcher.doc(doc_id)
    if doc == None:
        print("\tMissing")
        continue
    doc_text = doc.raw().lower()
    doc_text = re.sub("[^a-z0-9 ]", " ", doc_text)

    expansion_terms[query_id] = {}

    split_query_text = query_text.split()
    split_doc_text = doc_text.split()

    embeddings_query = model.encode(split_query_text, convert_to_tensor=True)
    embeddings_doc = model.encode(split_doc_text, convert_to_tensor=True)

    cosine_scores = util.cos_sim(embeddings_query, embeddings_doc)

    top_k = torch.topk(cosine_scores, k=considered_k, dim=1)

    for i,query_word in enumerate(split_query_text):
        expansion_terms[query_id][query_word] = {}
        top_doc_indices = top_k.indices[i]
        top_doc_scores = top_k.values[i]
        for j in range(considered_k):
            word_idx = top_doc_indices[j]
            exp_word = split_doc_text[word_idx]
            word_score = top_doc_scores[j].item()
            if word_score < upper_bound and word_score > lower_bound and len(exp_word) > 2:
                expansion_terms[query_id][query_word][exp_word] = word_score
                print("\t", query_word, exp_word)
            if len(expansion_terms[query_id][query_word]) > max_k: break

In [11]:
json.dump(expansion_terms, open("expanded_queries/" + "optimal_v2." + exp_queries + ".json", "w"))

#### Below is for optimal approach #1: keyword-based optimal expansion based on non-returned documents

In [16]:
# perform search, collect all document ids that ARE NOT returned by the query
# query_id : [relevant document ids not returned by search]

# missing_docids is map from query_id to list of doc_ids

missing_docids = {}
topk=1000
for query_id in all_query_words:
    missing_docids[query_id] = []
    query = all_query_words[query_id]
    rel_docs = set(all_qrels[query_id])
    hits = searcher.search(query, k=topk)
    hits_ids = set([hit.docid for hit in hits])
    missing_docids[query_id] = list(rel_docs.difference(hits_ids))


In [17]:
# collect doc text for all missing docs
# for all docs, make broad word index (map word to doc id occurrences)
# doc_id : doc text

all_needed = {} # map from docid to list of doc words
inverted_index = {} # map from word to docid

for x in missing_docids:
    for docid in missing_docids[x]:
        all_needed[docid] = None
print(len(all_needed))

for i,docid in enumerate(list(all_needed.keys())):
    doc = searcher.doc(docid)
    if doc == None:
        del all_needed[docid]
    else:
        doc_text = doc.raw()
        doc_text = doc_text.lower()
        doc_text = re.sub("[^a-z ]", " ", doc_text)
        doc_words = list(set([x for x in doc_text.split() if len(x) > 3]))
        all_needed[docid] = doc_words

        for word in doc_words:
            if word not in inverted_index:
                inverted_index[word] = []
            inverted_index[word].append(docid)

    if i % 10000 == 0: print(i)
print(len(all_needed))

4025
0
3818


For approach #3:
- Sart with relevant, non-matching
- Find words with high df > 30% of non-matching
- Select representative based on query 


In [18]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/kjros2/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [19]:
stopwords = {}
with open("stopwords.txt", "r") as f:
    stopwords = {word.strip("\n"): True for word in f.readlines()}

In [20]:
# missing_docids is map from query_id to list of doc_ids
# all_needed is map from docid to list of doc words
# inverted_index is map from word to docid
# all_query_words is map from query_id to raw query
"""
for each query
    for each doc id in the query
        add words to dict counter

    sort by most frequent
    compute similarity between most frequent and query
    once we hit threshold > 0.8? save to most similar term

"""
expansion_terms = {}
for query_id in missing_docids:

    query_text = all_query_words[query_id].lower()
    query_text = re.sub("[^a-z0-9 ]", " ", query_text)
    split_query = query_text.split()

    all_words = {}
    for doc_id in missing_docids[query_id]:
        for word in all_needed.get(doc_id, []):
            if word in stopwords: continue
            if word in split_query: continue
            all_words[word] = all_words.get(word, 0) + 1
    sorted_words = sorted([(word, all_words[word]) for word in all_words], reverse=True, key=lambda x: x[1])

    considered_k = 50
    top_words = [x[0] for x in sorted_words[:considered_k]]

    

    embeddings_query = model.encode(split_query, convert_to_tensor=True)
    embeddings_doc = model.encode(top_words, convert_to_tensor=True)

    cosine_scores = util.cos_sim(embeddings_query, embeddings_doc)

    top_k = torch.topk(cosine_scores, k=considered_k, dim=1)


    expansion_terms[query_id] = {}

    # 0. Fix stemming bug on bm25-s
    # 1. scatterplot of individual query scores
    # 2. normalize scores to sum to weight, redo experiments
    # 3. only expand along one query term


    for i,query_word in enumerate(split_query):
        expansion_terms[query_id][query_word] = {}
        top_doc_indices = top_k.indices[i]
        top_doc_scores = top_k.values[i]
        for j in range(considered_k):
            word_idx = top_doc_indices[j]
            exp_word = top_words[word_idx]
            word_score = top_doc_scores[j].item()
            if word_score < 0.5: continue
            if len(expansion_terms[query_id][query_word]) > 4: continue
            expansion_terms[query_id][query_word][exp_word] = word_score

    print(query_id)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [21]:
json.dump(expansion_terms, open("expanded_queries/" + "optimal_v3." + exp_queries + ".json", "w"))

In [ ]:
query_expansion_scores = {}
for query_id in missing_docids:
    rel_ids = {x: None for x in missing_docids[query_id] if x in all_needed}
    # start with words in relevant documents
    all_rel_words = {}
    for doc_id in rel_ids:
        for word in all_needed[doc_id]:
            all_rel_words[word] = {"rel_score": 0, "else_score": 0}

    for word in all_rel_words:
        rel_score = 0
        else_score = 0
        for occ_id in inverted_index[word]:
            if occ_id in rel_ids:
                all_rel_words[word]["rel_score"] += 1
            else:
                all_rel_words[word]["else_score"] += 1
    query_expansion_scores[query_id] = all_rel_words
    print(query_id)

In [39]:
# now, for each query, find the words with the highest ratio
expanded_queries = {}
for query_id in query_expansion_scores:
    # collect scores for each expansion word
    sorted_words = []
    for word in query_expansion_scores[query_id]:
        score = query_expansion_scores[query_id][word]["rel_score"] / (query_expansion_scores[query_id][word]["else_score"]+1)
        if query_expansion_scores[query_id][word]["rel_score"] > 100:
            sorted_words.append((word, score))
    sorted_words = sorted(sorted_words, reverse=True, key=lambda x: x[1])[:10]
    expanded_queries[query_id] = {"query": all_query_words[query_id], "expansion_terms": [x[0] for x in sorted_words]}
json.dump(expanded_queries, open("expanded_queries/optimal." + exp_queries + ".json", "w"))

In [43]:
# make various expansion datasets FOR PASSING DIRECTLY TO BM25
# weight 0, weight 1, weight 5, weight 10
expanded_queries = json.load(open("expanded_queries/optimal." + exp_queries + ".json", "r"))

weights = [0,1,5,10]

for weight in weights:
    lines = []
    with open(query_path, "r") as f:
        for line in f:
            lines.append(line)

    if exp_queries == "51-100":
        current_id = None
        for i,line in enumerate(lines):
            if "Number:" in line:
                current_id = str(int(line.split()[2]))
            if "Topic:" in line:
                line = line[:-1] # remove new line
                split_line = line.split()
                query = split_line[2:]
                meta_data = " ".join(split_line[0:2])
                line  = meta_data
                for j in range(weight):
                    line += " " + " ".join(query)
                query_expansion_terms = expanded_queries[current_id]["expansion_terms"]
                line += " " + " ".join(query_expansion_terms)
                lines[i] = line + "\n"
        with open("expanded_queries/optimal.disk12.bm25." + str(weight) + ".51-100.txt", "w") as f:
            for line in lines:
                f.write(line)

    elif exp_queries == "covid":
        for i,line in enumerate(lines):
            if "<topic number=" in line:
                current_id = re.sub("[^0-9]", "", line)
            if "<query>" in line:
                query = re.sub("<query>", "", line)
                query = re.sub("</query>\n", "", query)
                new_line = ""
                for j in range(weight):
                    new_line += query + " "
                query_expansion_terms = expanded_queries[current_id]["expansion_terms"]
                new_line += " " + " ".join(query_expansion_terms)
                lines[i] = "\t<query>" + new_line + "</query>\n"
        with open("expanded_queries/optimal.covid.bm25." + str(weight) + ".xml", "w") as f:
            for line in lines:
                f.write(line)
